In [1]:
import re
import math
import networkx as nx
from math import exp
from typing import List, Tuple, Set, Dict, Optional


In [2]:
def parse_txt_file(path: str):
    with open(path, "r") as f:
        lines = [ln.strip() for ln in f if ln.strip()]

    def extract_int(key):
        for ln in lines:
            if ln.upper().startswith(key.upper()):
                m = re.findall(r"(-?\d+)", ln)
                if m: return int(m[0])
        return None

    def extract_float(key):
        for ln in lines:
            if ln.upper().startswith(key.upper()):
                m = re.findall(r"(-?\d+(?:\.\d+)?)", ln)
                if m: return float(m[0])
        return None

    def extract_list_depots():
        for ln in lines:
            if ln.upper().startswith("DEPOT"):
                nums = re.findall(r"(\d+)", ln)
                return [int(x) for x in nums]
        return []

    nV = extract_int("NUMBER OF VERTICES")
    capacity = extract_float("VEHICLE CAPACITY") or 0.0
    depots = extract_list_depots()

    req_section = False
    nonreq_section = False
    required_edges = []
    weights = {}

    for ln in lines:
        if ln.upper().startswith("LIST_REQUIRED_EDGES"):
            req_section, nonreq_section = True, False
            continue
        if ln.upper().startswith("LIST_NON_REQUIRED_EDGES"):
            req_section, nonreq_section = False, True
            continue
        if ln.upper().startswith("FAILURE_SCENARIO"):
            req_section = nonreq_section = False
            continue

        m = re.match(r"\((\d+),\s*(\d+)\)\s*edge\s*weight\s*([0-9.]+)", ln, re.IGNORECASE)
        if m:
            u, v, w = int(m.group(1)), int(m.group(2)), float(m.group(3))
            wkey = tuple(sorted((u, v)))
            weights[wkey] = w
            if req_section:
                required_edges.append((u, v))

    G = nx.Graph()
    if nV:
        G.add_nodes_from(range(1, nV + 1))
    for (a, b), w in weights.items():
        G.add_edge(a, b, weight=w)

    return G, required_edges, depots, capacity


In [3]:
def edge_trip_feasible(G, depots, cap, a, b, dist):
    w_ab = G[a][b]['weight']
    best = float('inf'); best_tuple = None
    for s in depots:
        for e in depots:
            cost1 = dist[s][a] + w_ab + dist[b][e]  # traverse a->b
            cost2 = dist[s][b] + w_ab + dist[a][e]  # or b->a
            if cost1 < best:
                best, best_tuple = cost1, (s, 'a->b', e)
            if cost2 < best:
                best, best_tuple = cost2, (s, 'b->a', e)
    t1 = nx.shortest_path(G, source=best_tuple[0], target=a, weight='weight')
    t2 = nx.shortest_path(G, source=b, target=best_tuple[2], weight='weight')
    
    return best <= cap + 1e-9, best, best_tuple, t1 + t2


In [5]:
def run_intelligent_tuning_demo():

    for sce in range(21, 22):
        # print(f'\nRunning scenario - {sce}')
        # scenario_txt_path = f"gdb_failure_scenarios/gdb.{sce}.txt"   # change to your file name if needed
        # scenario_txt_path = f"bccm_failure_scenarios/bccm.{sce}.txt"   # change to your file name if needed
        scenario_txt_path = f"eglese_failure_scenarios/eglese.{sce}.txt"   # change to your file name if needed
        G, req_edges, depots, cap = parse_txt_file(scenario_txt_path)
        dist = dict(nx.all_pairs_dijkstra_path_length(G, weight='weight'))
        req_set_remaining = set(tuple(sorted(e)) for e in req_edges)
        START_DEPOT = depots[-1]
        MAX_VEHICLE_CAPACITY = cap
        for i in range(len(depots) - 1):
            for j in range(i+1, len(depots)):
                dist = nx.shortest_path_length(G=G, source=depots[i], target=depots[j], weight='weight')
                print(f'Distance b/w depot {depots[i]} and {depots[j]} - {dist}, vehicle capacity - {cap}')
		# print(cap)
        # ok1, lb1, trip1, t1 = edge_trip_feasible(G, depots, cap, 5, 6, dist)
        # ok2, lb2, trip2, t2 = edge_trip_feasible(G, depots, cap, 6, 12, dist)
        # print(f"(5,6): feasible={ok1}, best_lb={lb1:.2f}, witness={trip1}, trip - {t1}")
        # print(f"(6,12): feasible={ok2}, best_lb={lb2:.2f}, witness={trip2} trip - {t2}")
        # print(G[5][6]['weight'])
        # print(dist[16][5])
        # for d in depots:
        #     print(f'distace between 6 - {d} - {dist[6][d]}')
if __name__ == "__main__":
    run_intelligent_tuning_demo()


Distance b/w depot 2 and 5 - 73.0, vehicle capacity - 184.0
Distance b/w depot 2 and 11 - 155.0, vehicle capacity - 184.0
Distance b/w depot 2 and 17 - 268.0, vehicle capacity - 184.0
Distance b/w depot 2 and 19 - 224.0, vehicle capacity - 184.0
Distance b/w depot 2 and 31 - 296.0, vehicle capacity - 184.0
Distance b/w depot 2 and 33 - 344.0, vehicle capacity - 184.0
Distance b/w depot 2 and 35 - 249.0, vehicle capacity - 184.0
Distance b/w depot 2 and 43 - 209.0, vehicle capacity - 184.0
Distance b/w depot 2 and 58 - 124.0, vehicle capacity - 184.0
Distance b/w depot 2 and 60 - 157.0, vehicle capacity - 184.0
Distance b/w depot 2 and 72 - 315.0, vehicle capacity - 184.0
Distance b/w depot 5 and 11 - 82.0, vehicle capacity - 184.0
Distance b/w depot 5 and 17 - 195.0, vehicle capacity - 184.0
Distance b/w depot 5 and 19 - 161.0, vehicle capacity - 184.0
Distance b/w depot 5 and 31 - 290.0, vehicle capacity - 184.0
Distance b/w depot 5 and 33 - 378.0, vehicle capacity - 184.0
Distance b/